In [1]:
#import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

In [2]:
# device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Running on: {device}...")

Running on: cuda...


In [3]:
# Hyper parameter
batch_size=20
num_epochs=50
lr=1e-4
class_size=10
# percentage of training set to use as validation
valid_size = 0.2
# number of subprocesses to use for data loading
num_workers = 0

### Prepare dataset

In [4]:
transform_train = transforms.Compose([transforms.Resize((224,224)),                                     
                                     transforms.RandomHorizontalFlip(), 
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                          std=[0.229, 0.224, 0.225])])
transform_test = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(), 
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])])

#prep the train, validation and test dataset

train_data = torchvision.datasets.CIFAR10("data/", train=True, 
                                     download=True, 
                                     transform=transform_train) 

combined_test_data = torchvision.datasets.CIFAR10("data/", train=False, 
                                    download=True, 
                                    transform=transform_test) 

# Get targets from the test data for stratification
targets = combined_test_data.targets

# Stratified split for validation and test sets from the original test dataset
sss_test_valid = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)  # Split 50/50
valid_idx, test_idx = next(sss_test_valid.split(np.zeros(len(targets)), targets))

# Convert indices to actual Subset for validation and test datasets
valid_data = Subset(combined_test_data, valid_idx)
test_data = Subset(combined_test_data, test_idx)


#  train, val and test datasets to the dataloader
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

### Model Definition

In [5]:
class VGG16_NET(nn.Module):
    def __init__(self):
        super(VGG16_NET, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.conv8 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.conv11 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc14 = nn.Linear(25088, 4096)
        self.fc15 = nn.Linear(4096, 4096)
        self.fc16 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.maxpool(x)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.maxpool(x)
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))
        x = self.maxpool(x)
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.maxpool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc14(x))
        x = F.dropout(x, 0.5) #dropout was included to combat overfitting
        x = F.relu(self.fc15(x))
        x = F.dropout(x, 0.5)
        x = self.fc16(x)
        return x

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
VGG16 = VGG16_NET() 


VGG16.load_state_dict(torch.load('VGG16_20250420.pt', map_location=device))
VGG16.to(device)
print("Model loaded successfully")

Model loaded successfully


In [7]:
# Function to create a random patch
def create_patch(patch_size=(3, 3)):
    patch = torch.randn(3, *patch_size, requires_grad=True, device=device)
    return patch
# Function to apply the patch to images
def place_patch(img, patch):
    batch_size, _, h, w = img.size()
    ph, pw = patch.size(1), patch.size(2)
    for i in range(batch_size):
        x_offset = torch.randint(0, h - ph + 1, (1,)).item()
        y_offset = torch.randint(0, w - pw + 1, (1,)).item()
        img[i, :, x_offset:x_offset+ph, y_offset:y_offset+pw] = patch
    return img
# Function to evaluate the success rate of the adversarial patch
def evaluate_patch(model, patch, dataloader, target_class=None):
    model.eval()
    success = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            patched_images = place_patch(images, patch)
            outputs = model(patched_images)
            _, predicted = outputs.max(1)
            if target_class is not None:
                success += (predicted == target_class).sum().item()
            else:
                success += (predicted != labels).sum().item()  # Evaluate untargeted attack
            total += labels.size(0)

    successR = 100 * success / total
    print(f"Attack Success Rate: {successR:.2f}%")
    return successR

In [8]:
# CIFAR-10 class names
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Different patch sizes
patch_sizes = [(3, 3), (5, 5), (7, 7), (16, 16)]

# Evaluate patches of different sizes
for patch_size in patch_sizes:
    
    print(f"\nEvaluating untargeted adversarial patch, size: {patch_size}")
    patch = create_patch(patch_size)
    patch = torch.load(f'adversarial_patch_{patch_size[0]}x{patch_size[1]}_untargeted.pth')
    # Evaluate attack success rate
    evaluate_patch(VGG16, patch, test_loader, target_class=None)

    


Evaluating untargeted adversarial patch, size: (3, 3)


Attack Success Rate: 20.66%

Evaluating untargeted adversarial patch, size: (5, 5)
Attack Success Rate: 20.24%

Evaluating untargeted adversarial patch, size: (7, 7)
Attack Success Rate: 23.56%

Evaluating untargeted adversarial patch, size: (16, 16)
Attack Success Rate: 30.18%


In [9]:
# Evaluate patches of different sizes
for patch_size in patch_sizes:
    
    print(f"\nEvaluating targeted adversarial patch (car), size: {patch_size}")
    patch = create_patch(patch_size)
    patch = torch.load(f'adversarial_patch_{patch_size[0]}x{patch_size[1]}_targeted_car.pth')
    # Evaluate attack success rate
    evaluate_patch(VGG16, patch, test_loader, target_class=1)



Evaluating targeted adversarial patch (car), size: (3, 3)
Attack Success Rate: 10.42%

Evaluating targeted adversarial patch (car), size: (5, 5)
Attack Success Rate: 11.04%

Evaluating targeted adversarial patch (car), size: (7, 7)
Attack Success Rate: 10.80%

Evaluating targeted adversarial patch (car), size: (16, 16)
Attack Success Rate: 11.62%
